In [ ]:
!pip install datasets regex  -Uqqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 434.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from regex import Regex

In [ ]:
from datasets import load_dataset

import regex
from tokenizers import Tokenizer
from tokenizers.models import BPE, Unigram, WordLevel, WordPiece

from tokenizers.trainers import BpeTrainer, WordLevelTrainer, WordPieceTrainer, UnigramTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing

from transformers import AutoTokenizer
from tokenizers import normalizers, pre_tokenizers, processors, decoders

In [ ]:
from datasets import load_dataset

ds = load_dataset("Salesforce/wikitext", "wikitext-103-v1")
ds

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 1801350
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [ ]:
ds = ds.filter(lambda example: len(example['text']) > 0)

print("remove empty rows", ds)


def dedup_func(x):
    """Use this function to remove duplicate entries"""
    if x['text'] in unique_text:
        return False
    else:
        unique_text.add(x['text'])
        return True

unique_text = set()

ds = ds.filter(dedup_func, load_from_cache_file=False, num_proc=1)
unique_text.clear()

print("remove duplicate", ds)



import re

def clean_text(text):
      text = text.replace("@-@", "")
      text = text.replace("@.@", ".")
      text = text.replace("@,@", ",")
      text = text.replace("<unk>", "[UNK]")
      text = text.replace("\n", "")
      text = text.replace("\'", "")
      text = text.replace("\\", "")
      text = text.replace(" '", "'")



      return text


ds = ds.map(lambda example: {'text': clean_text(example['text'])})
ds


print("clean text", ds)


Filter:   0%|          | 0/4358 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3760 [00:00<?, ? examples/s]

remove empty rows DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 2891
    })
    train: Dataset({
        features: ['text'],
        num_rows: 1165029
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 2461
    })
})


Filter:   0%|          | 0/2891 [00:00<?, ? examples/s]

...

remove duplicate DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 2759
    })
    train: Dataset({
        features: ['text'],
        num_rows: 970883
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 2074
    })
})


Map:   0%|          | 0/2759 [00:00<?, ? examples/s]

Map:   0%|          | 0/970883 [00:00<?, ? examples/s]

Map:   0%|          | 0/2074 [00:00<?, ? examples/s]

clean text DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 2759
    })
    train: Dataset({
        features: ['text'],
        num_rows: 970883
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 2074
    })
})


In [ ]:
ds.keys()

dict_keys(['test', 'train', 'validation'])

In [ ]:
tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

tokenizer.normalizer = normalizers.Sequence(
    [
        normalizers.Replace(r"[\p{Other}&&[^\n\t\r]]", ""),
        normalizers.Replace(r"[\s]", " "),
        #normalizers.Lowercase(),
        normalizers.NFD(), normalizers.StripAccents()]
)

tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

special_tokens = ["[UNK]", "<|EOS|>"]
trainer = WordPieceTrainer(vocab_size=40000,  special_tokens=special_tokens)


for split in ds.keys():
    tokenizer.train_from_iterator(ds[split]['text'], trainer=trainer)

eos_token_id = tokenizer.token_to_id("<|EOS|>")

tokenizer.post_processor = processors.TemplateProcessing(

    single="<|EOS|> $A <|EOS|>",

    special_tokens=[ ("<|EOS|>", eos_token_id)],

)

In [ ]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [ ]:
tokenizer.get_vocab_size()

28144

In [ ]:
def tokenization(example):
    tokens = tokenizer.encode(example["text"])

    token_ids = tokens.ids

    

    example["input_ids"] = token_ids

    example["num_tokens"] = len(token_ids)

    return example

In [ ]:
ds = ds.map(tokenization, load_from_cache_file=False)
print(ds)

Map:   0%|          | 0/2759 [00:00<?, ? examples/s]

Map:   0%|          | 0/970883 [00:00<?, ? examples/s]

Map:   0%|          | 0/2074 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['text', 'input_ids', 'num_tokens'],
        num_rows: 2759
    })
    train: Dataset({
        features: ['text', 'input_ids', 'num_tokens'],
        num_rows: 970883
    })
    validation: Dataset({
        features: ['text', 'input_ids', 'num_tokens'],
        num_rows: 2074
    })
})


In [ ]:
ds['train']['input_ids'][0]

In [ ]:
text = """Prior to marrying his wife Valerie , John dated Carolyn Bessette the future wife of John F. Kennedy , Jr . The two met while attending University in Boston .
"""


output = tokenizer.encode(text)

print(output.ids)
print(len(output.ids))
print(tokenizer.decode(output.ids,skip_special_tokens = False))
#print(len(outptut.ids))

[1, 4823, 201, 26737, 294, 3215, 3857, 181, 366, 11, 1277, 6003, 2699, 1526, 9439, 3022, 180, 4945, 3215, 192, 1277, 37, 13, 12697, 11, 7663, 13, 217, 420, 1142, 618, 8841, 1497, 193, 6899, 13, 1]
37
<|EOS|> Prior to marrying his wife Valerie, John dated Carolyn Bessette the future wife of John F. Kennedy, Jr. The two met while attending University in Boston. <|EOS|>


In [ ]:
train_input_ids = np.concatenate(ds['train']["input_ids"])
print(len(train_input_ids))

import datasets

input_ids_list = train_input_ids.tolist()
packaged_pretrain_dataset = datasets.Dataset.from_dict(
    {"input_ids": train_input_ids}
)
print(packaged_pretrain_dataset)


packaged_pretrain_dataset.to_parquet("/content/sales_wordpiece_cap.parquet")

111902437
Dataset({
    features: ['input_ids'],
    num_rows: 111902437
})


Creating parquet from Arrow format:   0%|          | 0/111903 [00:00<?, ?ba/s]

447609748

In [ ]:
text = """Prior to marrying his wife Valerie , John dated Carolyn Bessette the future wife of John F. Kennedy , Jr . The two met while attending University in Boston .
"""

text = clean_text(text)
text
outptut = tokenizer.encode(text)

print(outptut.tokens)
print(outptut.ids)
print(len(outptut.ids))

print(tokenizer.decode(outptut.ids))

['<|BOS|>', 'prior', 'to', 'marrying', 'his', 'wife', 'val', '##erie', ',', 'john', 'dated', 'carol', '##yn', 'bess', '##ette', 'the', 'future', 'wife', 'of', 'john', 'f', '.', 'kennedy', ',', 'jr', '.', 'the', 'two', 'met', 'while', 'attending', 'university', 'in', 'boston', '.', '<|EOS|>']
[1, 2020, 150, 23082, 257, 3017, 1966, 16462, 12, 1220, 5487, 2545, 1448, 8477, 2838, 130, 4533, 3017, 143, 1220, 40, 14, 11278, 12, 6943, 14, 130, 358, 1140, 532, 7165, 1247, 140, 6297, 14, 2]
36
prior to marrying his wife val ##erie , john dated carol ##yn bess ##ette the future wife of john f . kennedy , jr . the two met while attending university in boston .


In [ ]:
tokenizer.decoder = decoders.WordPiece(prefix="##")


text = """Prior to marrying his wife Valerie , John dated Carolyn Bessette the future wife of John F. Kennedy , Jr . The two met while attending University in Boston .
"""

text = clean_text(text)
text
outptut = tokenizer.encode(text)

print(outptut.tokens)
print(outptut.ids)
print(len(outptut.ids))

print(tokenizer.decode(outptut.ids))

['<|BOS|>', 'prior', 'to', 'marrying', 'his', 'wife', 'val', '##erie', ',', 'john', 'dated', 'carol', '##yn', 'bess', '##ette', 'the', 'future', 'wife', 'of', 'john', 'f', '.', 'kennedy', ',', 'jr', '.', 'the', 'two', 'met', 'while', 'attending', 'university', 'in', 'boston', '.', '<|EOS|>']
[1, 2020, 150, 23082, 257, 3017, 1966, 16462, 12, 1220, 5487, 2545, 1448, 8477, 2838, 130, 4533, 3017, 143, 1220, 40, 14, 11278, 12, 6943, 14, 130, 358, 1140, 532, 7165, 1247, 140, 6297, 14, 2]
36
prior to marrying his wife valerie, john dated carolyn bessette the future wife of john f. kennedy, jr. the two met while attending university in boston.


In [ ]:

tokenizer.save("/content/sales_wordpiece_cap.json")